# For Animating Concentration Evolution under Dirchlet, Neumann, and Robin BCs 

## Animating Conc + Pos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Video, Image
from pathlib import Path

# ====== CONFIG (edit) ======
j = 15                 # y-slice index to visualize
Lx = 0.002
Ly = 0.0006
Lz = 0.002  # box lengths (m)
cmap = "viridis"
dot_size = 10
alpha = 0.8
interval_ms = 50
preview_downsample = 5   # reduce to avoid huge inline embeds
fix_colors = True        # keep vmin/vmax constant across frames

# Saving (optional)
save_mp4 = False        # set True if ffmpeg is installed
save_gif = False
mp4_name = "conc_positions.mp4"
gif_name = "conc_positions.gif"
fps = 20
dpi = 120
# ===========================

# Example relative paths (adjust as needed)
project_root = Path.cwd().parents[0]
C_PATH  = project_root / "src" / "n1000_t1000_chist_rev_irrevbind_515_mag0.npy"
POS_PATH = project_root / "src" / "n1000_t1000_pos_rev_irrevbind_515_mag0.npy"

# ---- Load arrays ----
C = np.load(str(C_PATH))            # (T, Ny, Nx, Nz)
pos = np.load(str(POS_PATH))        # (T, n_swimmers, 3)
print("C shape:", C.shape, "positions shape:", pos.shape)

# Basic checks & alignment
T, Ny, Nx, Nz = C.shape
T_pos, n_swimmers, _ = pos.shape
assert T == T_pos, f"Time mismatch: C has {T} frames, positions has {T_pos}"

frames = list(range(0, T, max(1, preview_downsample)))

def to_idx(coords, L, N):
    idx = np.floor(coords / L * N).astype(np.int64)
    return np.clip(idx, 0, N - 1)

# Fixed color scale (recommended)
if fix_colors:
    vmin = float(np.min(C[:, j, :, :]))
    vmax = float(np.max(C[:, j, :, :]))
else:
    vmin = vmax = None
print(f"Color scale vmin={vmin}, vmax={vmax}")

# Figure
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(C[frames[0], j, :, :].T, origin="lower",
               cmap=cmap, vmin=vmin, vmax=vmax, aspect="auto")
cbar = fig.colorbar(im, ax=ax); cbar.set_label("Oxygen (a.u.)")

scat = ax.scatter([], [], s=dot_size, c="red", edgecolors="none", alpha=alpha)
ax.set_xlabel("x index"); ax.set_ylabel("z index")

def update(f):
    im.set_data(C[f, j, :, :].T)
    xyz = pos[f, :, :]
    ix = to_idx(xyz[:, 0], Lx, Nx)
    iz = to_idx(xyz[:, 2], Lz, Nz)
    scat.set_offsets(np.column_stack([ix, iz]))
    ax.set_title(f"y-slice j={j}, frame {f}/{T-1}  (showing {n_swimmers} swimmers)")
    return im, scat

ani = FuncAnimation(fig, update, frames=frames, interval=interval_ms, blit=False, repeat=False)

# Inline preview (for big T, consider saving to file)
HTML(ani.to_jshtml())

if save_mp4:
    try:
        ani.save(mp4_name, fps=fps, dpi=dpi)
        display(Video(mp4_name, embed=True))
    except Exception as e:
        print("MP4 save failed (need ffmpeg?). Error:", e)

if save_gif:
    try:
        ani.save(gif_name, fps=fps)
        display(Image(filename=gif_name))
    except Exception as e:
        print("GIF save failed. Error:", e)


C shape: (5001, 30, 100, 100) positions shape: (5001, 1000, 3)
Color scale vmin=5.0, vmax=70.0


Animation size has reached 21051836 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.


## Animating Pos only

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Video, Image

# ====== CONFIG (edit) ======
# Box lengths (m) — must match your sim
Lx = 0.003
Ly = 0.003
Lz = 0.003

# Slice settings (only used if filter_by_slice=True)
j = 15           # virtual y-slice index to visualize
Ny = 32          # number of virtual slices along y (choose any reasonable value)
filter_by_slice = False  # set True to only show swimmers in slice j

# Viz
dot_size = 14
alpha = 0.9
interval_ms = 50
preview_downsample = 1    # show every Nth frame in notebook (increase if it’s large)

# Saving (optional)
save_mp4 = True           # requires ffmpeg installed
save_gif = False
mp4_name = "swimmers_slice.mp4"
gif_name = "swimmers_slice.gif"
fps = 20
dpi = 150
# ===========================

from pathlib import Path
POS_PATH = str(Path.cwd().parents[0] / "data" / "RR" / "sink1_5" / "pos" / "n1000_t1000_pos_RR_bind515_mag0.npy")

# ---- Load positions: pos shape (T, n_swimmers, 3) in meters ----
pos = np.load(POS_PATH)
T, n_swimmers, _ = pos.shape
frames = list(range(0, T, max(1, preview_downsample)))

# Compute y-slice bounds if filtering
if filter_by_slice:
    dy = Ly / Ny
    y_lo = j * dy
    y_hi = (j + 1) * dy
else:
    y_lo, y_hi = None, None

# Figure
fig, ax = plt.subplots(figsize=(6, 5))
scat = ax.scatter([], [], s=dot_size, c="red", edgecolors="none", alpha=alpha)

ax.set_xlim(0.0, Lx)
ax.set_ylim(0.0, Lz)
ax.set_xlabel("x (m)")
ax.set_ylabel("z (m)")

def update(f):
    xyz = pos[f]  # (n_swimmers, 3)
    if filter_by_slice:
        mask = (xyz[:, 1] >= y_lo) & (xyz[:, 1] < y_hi)
        xyz = xyz[mask]
        shown = xyz.shape[0]
        title_slice = f" | y-slice j={j}/{Ny-1}"
    else:
        shown = xyz.shape[0]
        title_slice = " | all swimmers (projected)"

    if shown == 0:
        scat.set_offsets(np.empty((0, 2)))
    else:
        scat.set_offsets(np.column_stack([xyz[:, 0], xyz[:, 2]]))

    ax.set_title(f"Frame {f}/{T-1}{title_slice} | showing {shown} swimmers")
    return (scat,)

ani = FuncAnimation(fig, update, frames=frames, interval=interval_ms, blit=False, repeat=False)

# Inline preview
HTML(ani.to_jshtml())

# Optional: save to file
if save_mp4:
    try:
        ani.save(mp4_name, fps=fps, dpi=dpi)
        display(Video(mp4_name, embed=True))
    except Exception as e:
        print("MP4 save failed (need ffmpeg?). Error:", e)

if save_gif:
    try:
        ani.save(gif_name, fps=fps)
        display(Image(filename=gif_name))
    except Exception as e:
        print("GIF save failed. Error:", e)


## Mean Pos

In [ ]:
time_steps = pos.shape[0]
t = range(time_steps)

avg_z = pos[:, :, 2].mean(axis=1)
std_z = pos[:, :, 2].std(axis=1, ddof=1)

plt.figure(figsize=(8,5))
plt.plot(t, avg_z, label=r"$\langle z \rangle$", color="C0")
plt.fill_between(t, avg_z - std_z, avg_z + std_z, color="C0", alpha=0.2, label="±1 std dev")
plt.xlabel("Time step")
plt.ylabel(r"$\langle z \rangle$")
plt.title(r"$\langle z \rangle$" + " | n_swimmers=1000, t=1000, box=0.002x0.0006x0.002 m")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
time_steps = pos.shape[0]
t = range(time_steps)

avg_z = pos[:, :, 0].mean(axis=1)
std_z = pos[:, :, 0].std(axis=1, ddof=1)

plt.figure(figsize=(8,5))
# plot mean
plt.plot(t, avg_z, label=r"$\langle x \rangle$", color="C0")
# shaded std band
plt.fill_between(
    t,
    avg_z - std_z,
    avg_z + std_z,
    color="C0",
    alpha=0.2,
    label="±1 std dev"
)

plt.xlabel("Time step")
plt.ylabel(r"$\langle x \rangle$")
plt.title(r"$\langle x \rangle$" + "for simulation with n_swimmers = 1000, t = 1000")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
time_steps = pos.shape[0]
t = range(time_steps)

avg_y = pos[:, :, 1].mean(axis=1)
std_y = pos[:, :, 1].std(axis=1, ddof=1)

plt.figure(figsize=(8,5))
# plot mean
plt.plot(t, avg_y, label=r"$\langle y \rangle$", color="C0")
# shaded std band
plt.fill_between(
    t,
    avg_y - std_y,
    avg_y + std_y,
    color="C0",
    alpha=0.2,
    label="±1 std dev"
)

plt.xlabel("Time step")
plt.ylabel(r"$\langle y \rangle$")
plt.title(r"$\langle y \rangle$" + "for simulation with n_swimmers = 1000, t = 1000")
plt.legend()
plt.tight_layout()
plt.show()